# **전처리**
1. 초기 전처라
2. 키워드 정제 및 카테고리 분류
3. 중복 제거 및 고유 게임 파악
4. [지점별 보유 게임] 테이블
5. [게임 정보] 테이블
6. [지점 정보] 테이블

## **0. 라이브러리 호출**

In [3]:
import pandas as pd
import numpy as np
import re

## **1. 초기 전처리**
- CSV 파일 로딩
- 결측치/이상치 처리
- 지점명 정제

In [5]:
# 데이터 불러오기 및 기본 정제
df = pd.read_csv('redbutton_web_data_crawling_result.csv', encoding='cp949')
df.loc[df['게임명'] == '핏츠', ['인원', '게임시간', '키워드']] = ['1-4인', '45분', '퍼즐']
df['지점'] = np.where(df['지점'].str.endswith('점', na=False), df['지점'].str[:-1], df['지점'])

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,키워드
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,셋컬렉션
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력
...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,"주사위, 경주"
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드 전략
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,"순발력, 명대사"
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림/마피아


<br>

## **2. 키워드 정제 및 카테고리 분류**
- 키워드 전처리 함수 정의
- 오탈자 수정 및 표준화 함수 정의
- 대표 키워드 추출 및 상위 카테고리 생성
- 상위 카테고리 매핑

In [7]:
# 키워드 전처리 함수 정의
def clean_and_split_keywords(keyword):
    if pd.isna(keyword):
        return []
    cleaned = re.sub(r'[#/]', '', keyword)
    cleaned = re.sub(r'[·|&]', ',', cleaned)
    cleaned = cleaned.replace(' ', '')
    split_keywords = re.split(r'[,]', cleaned)
    return [kw for kw in split_keywords if kw]

def fix_keyword(keyword):
    if pd.isna(keyword):
        return ''
    exact_match_map = {
        '엔진빌': '엔진빌딩',
        '이야기만들': '이야기',
        '순발': '순발력',
        '리얼타임순발력': '순발력',
        '복물복': '복불복',
        '힘대결': '카드',
        '미정': '퍼즐'
    }
    partial_match_map = {
        '컬랙션': '컬렉션',
        '콜랙션': '컬렉션',
        '배팅': '베팅',
        '미니어쳐': '미니어처',
        '셋': '세트',
        '머더미스터리': '머더'
    }
    if keyword in exact_match_map:
        return exact_match_map[keyword]
    for wrong, correct in partial_match_map.items():
        if wrong in keyword:
            keyword = keyword.replace(wrong, correct)
    return keyword

# 키워드 분리 및 카테고리 생성
df['분리된_키워드'] = df['키워드'].apply(clean_and_split_keywords)
df['대표_키워드'] = df['분리된_키워드'].apply(lambda x: x[0] if x else '')
df['카테고리'] = df['대표_키워드'].apply(fix_keyword)
df.drop(['키워드', '분리된_키워드', '대표_키워드'], axis=1, inplace=True)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력
...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아


In [8]:
# 상위카테고리 매핑
mapping_df = pd.read_csv('category_mapping_table.csv', encoding='cp949')
mapping_dict = dict(zip(mapping_df['하위카테고리'], mapping_df['상위카테고리']))
df['상위카테고리'] = df['카테고리'].map(mapping_dict)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리,상위카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션,전략-전쟁-경영
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력,순발력-손기술
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리,추리-마피아-심리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력,협력-팀플레이
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력,단어-퀴즈-숫자-지식
...,...,...,...,...,...,...,...,...,...
35563,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위,주사위
35564,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략,카드-덱빌딩
35565,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력,순발력-손기술
35566,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아,그림


## **3. 중복 제거 및 고유 게임 파악**
- 지역-지점-게임명 기준 중복 제거
- 고유 게임 수 계산

In [10]:
# 중복 제거 및 고유 게임 수 계산
cols = ['지역', '지점', '게임명']
df = df.drop_duplicates(subset=cols).reset_index(drop=True)

df

,지역,지점,게임명,난이도,인원,게임시간,설명영상,카테고리,상위카테고리
0,서울,신촌,세븐원더스 : 듀얼,Extreme,2인,60분,아니오,세트컬렉션,전략-전쟁-경영
1,서울,신촌,도블,Easy,2-5인,10분,예,순발력,순발력-손기술
2,서울,신촌,데어이쎄스,Easy,2인,5분,예,심리추리,추리-마피아-심리
3,서울,신촌,데드오브윈터,ProGamer,2-5인,100분,아니오,미션협력,협력-팀플레이
4,서울,신촌,더게임,Normal,2-5인,20분,예,숫자협력,단어-퀴즈-숫자-지식
...,...,...,...,...,...,...,...,...,...
35410,제주,제주시청,캣츠런,Very Easy,2-4인,10분,예,주사위,주사위
35411,제주,제주시청,풍기,Hard,2인,30분,아니오,카드전략,카드-덱빌딩
35412,제주,제주시청,나의 최애 명대사,Easy,3-6인,15분,아니오,순발력,순발력-손기술
35413,제주,제주시청,라이헌트 미스터리 아티스트,Easy,3-8인,30분,예,그림마피아,그림


<br>

## **4. [보유게임] 테이블**
- 중복 제거 및 불필요 컬럼 삭제
- 컬럼명 영문화


### **컬럼**
- 지점명(location)
- 게임 이름(gameName)

In [12]:
# 보유 게임 목록 생성
taken = df.drop_duplicates().drop(['지역', '난이도','인원','게임시간', '설명영상', '카테고리', '상위카테고리'], axis=1)
taken.rename(columns={'지점': 'location', '게임명': 'gameName'}, inplace=True)
taken.reset_index(drop=True, inplace=True)

taken

,location,gameName
0,신촌,세븐원더스 : 듀얼
1,신촌,도블
2,신촌,데어이쎄스
3,신촌,데드오브윈터
4,신촌,더게임
...,...,...
35410,제주시청,캣츠런
35411,제주시청,풍기
35412,제주시청,나의 최애 명대사
35413,제주시청,라이헌트 미스터리 아티스트


In [13]:
taken.to_csv('taken_table.csv', index=False)

<br>

## **5. 게임 정보 테이블**
- 중복 제거 및 게임 단위 정리
- 라벨 인코딩(난이도)
- 게임 시간 정제
- 최소/최대인원 추출
- 섦여영상 보우 여부 처리
- 이달의 게임 여부 추가
- 컬럼명 영문화

### **컬럼**
- 게임명(gameName)
- 난이도(gameLevel)
- 게임시간(gameTime)
- 설명영상(gameRule)
- 카테고리(gameCategory)
- 상위카테고리(parentCategory)
- 난이도_라벨링(gameLevel_Encoding)
- 최소인원(gamePlayerMin)
- 최대인원(gamePlayerMax)
- 이달의게임여부(monthlyGame)

In [15]:
# 게임 정보 테이블 생성

def extract_min_max(person_str):
    if pd.isna(person_str):
        return pd.Series([pd.NA, pd.NA], dtype='Int64')
    cleaned = person_str.replace('명', '').replace('인', '').replace(' ', '').replace('~', '-').replace(',', '-')
    numbers = [int(n) for n in re.findall(r'\d+', cleaned)]
    if not numbers:
        return pd.Series([pd.NA, pd.NA], dtype='Int64')
    if len(numbers) == 1:
        return pd.Series([numbers[0], numbers[0]], dtype='Int64')
    return pd.Series([min(numbers), max(numbers)], dtype='Int64')

gameinfo = df.drop_duplicates(subset=['게임명']).drop(['지역','지점'], axis=1).copy()
gameinfo['난이도'] = pd.Categorical(gameinfo['난이도'], categories=['Very Easy', 'Easy', 'Normal', 'Hard', 'Very Hard', 'Extreme', 'ProGamer'], ordered=True)
gameinfo['난이도_라벨링'] = gameinfo['난이도'].cat.codes
gameinfo['게임시간'] = gameinfo['게임시간'].str.replace('분', '', regex=False).astype(int)
gameinfo[['최소인원', '최대인원']] = gameinfo['인원'].apply(extract_min_max)
gameinfo['설명영상'] = gameinfo['설명영상'].map({'예': 1, '아니오': 0}).astype(bool)

monthly_games = ["라이헌트", "다빈치코드", "금지어 게임", "루미큐브", "라쿠카라차", "클루", "스플렌더", "선물입니다", "인디언포커", "꼬치의 달인"]
gameinfo['이달의게임여부'] = gameinfo['게임명'].apply(lambda x: x.strip().startswith("라이헌트") if x.strip().startswith("라이헌트") else x.strip() in monthly_games)

gameinfo.rename(columns={
    '게임명': 'gameName',
    '난이도': 'gameLevel',
    '게임시간': 'gameTime',
    '설명영상': 'gameRule',
    '카테고리': 'gameCategory',
    '상위카테고리': 'parentCategory',
    '난이도_라벨링': 'gameLevel_Encoding',
    '최소인원': 'gamePlayerMin',
    '최대인원': 'gamePlayerMax',
    '이달의게임여부': 'monthlyGame',
}, inplace=True)

gameinfo

,gameName,gameLevel,인원,gameTime,gameRule,gameCategory,parentCategory,gameLevel_Encoding,gamePlayerMin,gamePlayerMax,monthlyGame
0,세븐원더스 : 듀얼,Extreme,2인,60,False,세트컬렉션,전략-전쟁-경영,5,2,2,False
1,도블,Easy,2-5인,10,True,순발력,순발력-손기술,1,2,5,False
2,데어이쎄스,Easy,2인,5,True,심리추리,추리-마피아-심리,1,2,2,False
3,데드오브윈터,ProGamer,2-5인,100,False,미션협력,협력-팀플레이,6,2,5,False
4,더게임,Normal,2-5인,20,True,숫자협력,단어-퀴즈-숫자-지식,2,2,5,False
...,...,...,...,...,...,...,...,...,...,...,...
34156,마피아 넘버 세븐,Easy,3-10인,30,False,마피아,추리-마피아-심리,1,3,10,False
34159,판타스틱,Easy,3-6인,20,False,미니게임손기술,파티-가족-미니게임,1,3,6,False
34161,모노폴리 신용카드 결제 버전,Easy,3-5인,120,False,부동산투자,경제-거래-경매-베팅,1,3,5,False
34162,공룡섬,Extreme,2-4인,90,False,전략,전략-전쟁-경영,5,2,4,False


In [16]:
gameinfo.to_csv("gameinfo_table.csv", index=False)

## **6. [지점 정보] 테이블**
- 지점별 보유 게임 리스트 생성
- 보유/미보유 게임 수 계산
- 컬럼명 영문화

### **컬럼**
- 지역(area)
- 지점명(location)
- 보유 게임 개수(numOfTaken)
- 비보유 게임 개수(numOfUntaken)

In [18]:
# 지점별 보유 게임 통계 계산
stores = df.drop_duplicates().drop(['난이도','인원','게임시간','설명영상','카테고리'], axis=1)
stores = stores.groupby(['지역', '지점'], sort=False)['게임명'].apply(list).reset_index()
stores['보유게임수'] = stores['게임명'].apply(len)
unique_count = df[['게임명', '난이도', '인원', '게임시간', '설명영상', '카테고리']].drop_duplicates().shape[0]
stores['비보유게임수'] = unique_count - stores['보유게임수']
stores.drop(['게임명'], axis=1, inplace=True)

# 이달의 게임 보유 수 계산 및 병합
monthly_game_names = gameinfo.query("monthlyGame == True")["gameName"].str.strip().tolist()
taken["gameName"] = taken["gameName"].str.strip()
taken_monthly = taken[taken["gameName"].isin(monthly_game_names)]

monthly_counts = taken_monthly["location"].value_counts().reset_index()
monthly_counts.columns = ["location", "이달의게임수"]
monthly_counts.rename(columns={"location": "지점"}, inplace=True)

stores = stores.drop(columns=[col for col in stores.columns if "이달의게임수" in col], errors="ignore")
stores = stores.merge(monthly_counts, on="지점", how="left")
stores["이달의게임수"] = stores["이달의게임수"].fillna(0).astype(int)

stores.rename(columns={'지역': 'area', 
                       '지점': 'location', 
                       '보유게임수': 'numOfTaken', 
                       '비보유게임수': 'numOfUntaken',
                       '이달의게임수': 'numOfMonthlyGames'}, inplace=True)

stores

,area,location,numOfTaken,numOfUntaken,numOfMonthlyGames
0,서울,신촌,412,665,11
1,서울,신림,365,712,11
2,서울,대학로,424,653,11
3,서울,강남,406,671,11
4,서울,노원,346,731,11
...,...,...,...,...,...
99,부산,부산대,460,617,11
100,부산,부산사상,332,745,11
101,부산,광안리,300,777,11
102,부산,부산동래,322,755,11


In [19]:
stores.to_csv('stores_table.csv', index=False)